In [1]:
# Importando as bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import warnings

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
warnings.filterwarnings("ignore") 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Lendo o arquivo
df = pd.read_csv("Dados/dados_clientes.csv")

## Análise Exploratória

In [3]:
# Visualizando o tamanhoos  do dataframe
df.shape

(80143, 24)

In [34]:
# Visualizando as colunas
df.columns

Index(['Contrato', 'Idade', 'Sexo', 'Valor_Renda', 'UF_Cliente', 'Perc_Juros',
       'Prazo_Emprestimo', 'Data_Contratacao', 'Prazo_Restante',
       'VL_Emprestimo', 'VL_Emprestimo_ComJuros', 'QT_Total_Parcelas_Pagas',
       'QT_Total_Parcelas_Pagas_EmDia', 'QT_Total_Parcelas_Pagas_EmAtraso',
       'Qt_Renegociacao', 'Estado_Civil', 'Escolaridade', 'Possui_Patrimonio',
       'VL_Patrimonio', 'QT_Parcelas_Atraso', 'QT_Dias_Atraso',
       'Saldo_Devedor', 'Total_Pago', 'Possivel_Fraude'],
      dtype='object')

In [4]:
# Visualizando as primeiras linhas
df.head()

,Contrato,Idade,Sexo,Valor_Renda,UF_Cliente,Perc_Juros,Prazo_Emprestimo,Data_Contratacao,Prazo_Restante,VL_Emprestimo,VL_Emprestimo_ComJuros,QT_Total_Parcelas_Pagas,QT_Total_Parcelas_Pagas_EmDia,QT_Total_Parcelas_Pagas_EmAtraso,Qt_Renegociacao,Estado_Civil,Escolaridade,Possui_Patrimonio,VL_Patrimonio,QT_Parcelas_Atraso,QT_Dias_Atraso,Saldo_Devedor,Total_Pago,Possivel_Fraude
0,322090928715,42,M,4000.00,MA,17.00,140,2022-11-18,143,160000.00,184000.00,2,1,1,2,SOLTEIRO(A),NaN,N,0.00,10,284.00,187861.70,1617.36,Sim
1,321990634715,31,M,3000.00,MA,20.00,28,2021-07-23,0,14000.00,16100.00,2,1,1,1,CASADO (A),NaN,N,0.00,26,771.00,16615.93,1239.98,Sim
2,321965373715,36,F,2100.00,SP,24.00,180,2021-04-01,149,60000.00,69000.00,4,1,0,1,CASADO (A),Nenhum,N,0.00,27,802.00,74443.40,1346.64,Sim
3,321967133715,28,M,2155.00,DF,19.00,190,2021-04-10,159,180000.00,207000.00,29,28,0,2,SOLTEIRO(A),Nenhum,N,0.00,2,41.00,196812.41,22713.63,Nao
4,322098744715,21,F,4300.00,MG,22.00,100,2022-12-28,94,30000.00,34500.00,4,3,0,1,SOLTEIRO(A),Ensino Médio,N,0.00,6,162.00,36114.39,900.80,Sim


In [5]:
# Visualizando 10 linhas aleatórias
df.sample(10)

,Contrato,Idade,Sexo,Valor_Renda,UF_Cliente,Perc_Juros,Prazo_Emprestimo,Data_Contratacao,Prazo_Restante,VL_Emprestimo,VL_Emprestimo_ComJuros,QT_Total_Parcelas_Pagas,QT_Total_Parcelas_Pagas_EmDia,QT_Total_Parcelas_Pagas_EmAtraso,Qt_Renegociacao,Estado_Civil,Escolaridade,Possui_Patrimonio,VL_Patrimonio,QT_Parcelas_Atraso,QT_Dias_Atraso,Saldo_Devedor,Total_Pago,Possivel_Fraude
3150,321977355715,56,M,3131.00,PE,22.00,180,2021-05-27,159,180000.00,207000.00,6,4,1,3,VIÚVO(A),Nenhum,N,0.00,23,680.00,216825.00,3400.50,Sim
16790,322098018715,45,F,3500.00,SC,20.00,200,2022-12-23,198,100000.00,115000.00,7,2,4,2,CASADO (A),NaN,N,0.00,4,103.00,118578.61,2500.24,Nao
21794,321946593715,28,M,6000.00,BA,18.00,100,2021-01-06,79,25000.00,28750.00,1,1,0,1,SOLTEIRO(A),NaN,N,0.00,32,953.00,29638.55,297.50,Sim
4769,321971182715,66,M,4000.00,RJ,24.00,30,2021-04-29,4,3500.00,4025.00,26,16,8,2,CASADO (A),NaN,N,0.00,0,NaN,0.00,5042.60,Nao
67437,321961965715,26,M,6060.00,SP,18.00,150,2021-03-15,129,250000.00,287500.00,6,2,0,0,CASADO (A),Nenhum,N,0.00,26,771.00,288591.33,11901.00,Sim
36986,321976054715,26,M,2200.00,PA,16.80,60,2021-05-21,34,15000.00,17250.00,26,7,3,3,SOLTEIRO(A),Nenhum,N,0.00,0,NaN,0.00,10786.98,Nao
67764,322027740715,24,F,1400.00,MG,18.00,25,2022-01-17,7,12000.00,13800.00,19,2,17,3,SOLTEIRO(A),Nenhum,N,0.00,1,11.00,1872.15,13414.34,Nao
11238,322024607715,48,M,1500.00,MS,18.00,100,2021-12-13,89,25000.00,28750.00,11,11,0,0,OUTRO,NaN,N,0.00,12,344.00,26619.71,3298.51,Sim
20509,322053020715,21,M,1930.00,SP,21.00,40,2022-05-07,25,10000.00,11500.00,1,1,0,0,SOLTEIRO(A),Nenhum,N,0.00,17,498.00,12238.92,309.57,Sim
16982,321996314715,76,M,2000.00,BA,21.00,60,2021-08-16,37,8000.00,9200.00,1,1,0,0,SOLTEIRO(A),NaN,N,0.00,25,741.00,9841.00,164.00,Sim


In [12]:
# Visualizando o período dos dados
data_inicial = pd.to_datetime(df['Data_Contratacao']).dt.date.min().strftime('%d/%m/%Y')
data_final = pd.to_datetime(df['Data_Contratacao']).dt.date.max().strftime('%d/%m/%Y')
print('Período: ', data_inicial, ' até ', data_final)

Período:  25/11/2020  até  22/02/2023


In [15]:
# Visualizando as informações básicas do DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80143 entries, 0 to 80142
Data columns (total 24 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Contrato                          80143 non-null  int64  
 1   Idade                             80143 non-null  int64  
 2   Sexo                              80143 non-null  object 
 3   Valor_Renda                       80143 non-null  float64
 4   UF_Cliente                        80143 non-null  object 
 5   Perc_Juros                        80143 non-null  float64
 6   Prazo_Emprestimo                  80143 non-null  int64  
 7   Data_Contratacao                  80143 non-null  object 
 8   Prazo_Restante                    80143 non-null  int64  
 9   VL_Emprestimo                     80143 non-null  float64
 10  VL_Emprestimo_ComJuros            80143 non-null  float64
 11  QT_Total_Parcelas_Pagas           80143 non-null  int64  
 12  QT_T

In [18]:
# Visualizando as estatísticas
df.describe()

,Contrato,Idade,Valor_Renda,Perc_Juros,Prazo_Emprestimo,Prazo_Restante,VL_Emprestimo,VL_Emprestimo_ComJuros,QT_Total_Parcelas_Pagas,QT_Total_Parcelas_Pagas_EmDia,QT_Total_Parcelas_Pagas_EmAtraso,Qt_Renegociacao,VL_Patrimonio,QT_Parcelas_Atraso,QT_Dias_Atraso,Saldo_Devedor,Total_Pago
count,80143.00,80143.00,80143.00,80143.00,80143.00,80143.00,80143.00,80143.00,80143.00,80143.00,80143.00,80143.00,80143.00,80143.00,54348.00,80143.00,80143.00
mean,322023695416.17,38.82,29308.62,19.49,102.33,93.28,72847.78,83774.94,10.71,5.24,2.34,1.30,1.45,10.65,475.26,78622.32,9687.98
std,44697010.94,12.53,2748166.27,3.56,62.32,70.64,87407.93,100519.12,9.31,6.21,3.97,1.27,355.10,10.43,269.79,102318.95,19090.62
min,321944855715.00,4.00,0.00,4.49,14.00,-18.00,3000.00,3450.00,0.00,0.00,0.00,0.00,0.00,0.00,11.00,0.00,0.00
25%,321984967215.00,29.00,2050.00,18.00,55.00,36.00,15000.00,17250.00,2.00,1.00,0.00,0.00,0.00,0.00,284.00,12122.76,1052.16
50%,322024156715.00,37.00,3000.00,20.00,80.00,78.00,40000.00,46000.00,8.00,2.00,1.00,1.00,0.00,9.00,468.00,35489.36,3951.39
75%,322062662215.00,46.00,5000.00,21.00,170.00,156.00,100000.00,115000.00,18.00,7.00,3.00,2.00,0.00,20.00,708.00,117783.25,10886.24
max,322099706715.00,103.00,765000000.00,28.00,240.00,227.00,500000.00,575000.00,66.00,66.00,31.00,38.00,100000.00,34.00,1014.00,626261.27,465821.39


In [16]:
# Verificando se existem valores ausentes
df.isnull().sum()

Contrato                                0
Idade                                   0
Sexo                                    0
Valor_Renda                             0
UF_Cliente                              0
Perc_Juros                              0
Prazo_Emprestimo                        0
Data_Contratacao                        0
Prazo_Restante                          0
VL_Emprestimo                           0
VL_Emprestimo_ComJuros                  0
QT_Total_Parcelas_Pagas                 0
QT_Total_Parcelas_Pagas_EmDia           0
QT_Total_Parcelas_Pagas_EmAtraso        0
Qt_Renegociacao                         0
Estado_Civil                            0
Escolaridade                        57298
Possui_Patrimonio                       0
VL_Patrimonio                           0
QT_Parcelas_Atraso                      0
QT_Dias_Atraso                      25795
Saldo_Devedor                           0
Total_Pago                              0
Possivel_Fraude                   

In [21]:
# Verificando os valores únicos de cada coluna do DataFrame
unique_values = []
for i in df.columns[:24].tolist():
    print(i, ':', len(df[i].astype(str).value_counts()))
    unique_values.append(len(df[i].astype(str).value_counts()))

Contrato : 80143
Idade : 86
Sexo : 2
Valor_Renda : 4747
UF_Cliente : 27
Perc_Juros : 53
Prazo_Emprestimo : 60
Data_Contratacao : 751
Prazo_Restante : 93
VL_Emprestimo : 73
VL_Emprestimo_ComJuros : 73
QT_Total_Parcelas_Pagas : 50
QT_Total_Parcelas_Pagas_EmDia : 48
QT_Total_Parcelas_Pagas_EmAtraso : 32
Qt_Renegociacao : 15
Estado_Civil : 8
Escolaridade : 6
Possui_Patrimonio : 2
VL_Patrimonio : 4
QT_Parcelas_Atraso : 35
QT_Dias_Atraso : 39
Saldo_Devedor : 64166
Total_Pago : 54074
Possivel_Fraude : 2


In [26]:
# Identificando possíveis outliers explícitos
print('Maior Idade: ', df['Idade'].max())
print('Menor Idade: ', df['Idade'].min())
print('----------------------------------------------------')
print('Maior Renda:', df['Valor_Renda'].max())
print('Menor Renda:', df['Valor_Renda'].min())
print('----------------------------------------------------')
print('Maior Juros:', df['Perc_Juros'].max())
print('Menor Juros:', df['Perc_Juros'].min())
print('----------------------------------------------------')
print('Maior qtd de dias restante: ', df['Prazo_Restante'].max())
print('Menor qtd de dias restante: ', df['Prazo_Restante'].min())
print('----------------------------------------------------')
print('Maior qtd de parcelas em atraso: ', df['QT_Parcelas_Atraso'].max())
print('Menor qtd de parcelas em atraso: ', df['QT_Parcelas_Atraso'].min())
print('----------------------------------------------------')
print('Maior qtd de dias de atraso: ', df['QT_Dias_Atraso'].max())
print('Menor qtd de dias de atraso: ', df['QT_Dias_Atraso'].min())
print('----------------------------------------------------')


Maior Idade:  103
Menor Idade:  4
----------------------------------------------------
Maior Renda: 765000000.0
Menor Renda: 0.0
----------------------------------------------------
Maior Juros: 28.0
Menor Juros: 4.4931
----------------------------------------------------
Maior qtd de dias restante:  227
Menor qtd de dias restante:  -18
----------------------------------------------------
Maior qtd de parcelas em atraso:  34
Menor qtd de parcelas em atraso:  0
----------------------------------------------------
Maior qtd de dias de atraso:  1014.0
Menor qtd de dias de atraso:  11.0
----------------------------------------------------


In [40]:
# Filtrando clientes menores de 18 anos
clientes_menores_18 = df[df['Idade'] < 18]
clientes_menores_18.groupby(['Idade']).size()


Idade
4      2
6      1
7      1
8      1
9      1
10     1
17    12
dtype: int64

In [41]:
# Filtrando clientes maiores de 85 anos
clientes_maiores_85 = df[df['Idade'] > 85]
clientes_maiores_85.groupby(['Idade']).size()

Idade
86     9
87     5
88     7
89     3
90     3
91     1
92     1
94     1
97     1
99     1
103    1
dtype: int64